In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
class MnistCustomDataset(Dataset):
  def __init__(self, csv_file, transform=None):
    self.transform = transform
    self.data = pd.read_csv(csv_file)
    self.targets = torch.tensor(self.data.loc[:, "label"].to_numpy())
  
  def __getitem__(self, index):
    sample = self.data.loc[index][1:].to_numpy(dtype=np.float32).reshape(28,28)
    target = torch.tensor(self.data.loc[index][0])
    if self.transform:
      sample = self.transform(sample)
    return sample, target

  def __len__(self):
    return self.data.shape[0]

In [3]:
# Calculating the mean and standard deviation of the dataset.

train_set = MnistCustomDataset(csv_file="/content/drive/MyDrive/Deep-Learning-Revision/mnist_dataset/fashion_mnist/fashion-mnist_train.csv",
                               transform = transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(train_set, batch_size=1024)


total_sum = 0
num_pixels = len(train_set) * 28 * 28
for image, _ in train_loader: total_sum += image.sum()
mean = total_sum/num_pixels

sum_squared_error = 0
for image, _ in train_loader: sum_squared_error += ((image-mean).pow(2)).sum()
std = torch.sqrt(sum_squared_error/num_pixels)
print(f"Mean: {mean} \t Std: {std}")

Mean: 72.95682525634766 	 Std: 89.96687316894531


In [4]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean, std)
                                      ])

In [5]:
train_set = MnistCustomDataset(csv_file="/content/drive/MyDrive/Deep-Learning-Revision/mnist_dataset/fashion_mnist/fashion-mnist_train.csv",
                               transform = transformations)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

    self.fc1 = nn.Linear(in_features = 12*4*4, out_features = 128)
    self.fc2 = nn.Linear(in_features = 128, out_features = 64)
    self.out = nn.Linear(in_features = 64, out_features = 10)

  def forward(self, t):
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    t = t.reshape(-1, 12*4*4) # flatten the tensor

    t = self.fc1(t)
    t = F.relu(t)

    t = self.fc2(t)
    t = F.relu(t)

    t = self.out(t)

    return t

In [24]:
network = Network()
network.to(device)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)

In [25]:
network.to(device)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)

In [26]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import time
from IPython.display import clear_output
import json
import pandas as pd

In [27]:
class RunBuilder():
  @staticmethod
  def get_runs(params):
    Run = namedtuple('Run', params.keys())
    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    return runs

In [28]:
class RunManager():
  def __init__(self):
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_start_time = None

    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    self.network = None
    self.loader = None
  
  def begin_run(self, run, network, loader):
    self.run_start_time = time.time()
    self.run_params = run
    self.run_count += 1
    self.network = network
    self.loader = loader
  
  def end_run(self):
    self.epoch_count = 0

  def begin_epoch (self):
    self.epoch_start_time = time.time()
    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0
  
  def end_epoch(self):
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)


    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["loss"] = loss
    results["accuracy"] = accuracy
    results["epoch_duration"] = epoch_duration
    results["run_duration"] = run_duration

    for k, v in self.run_params._asdict().items():
      results[k] = v

    self.run_data.append(results)

    df = pd.DataFrame.from_dict(self.run_data, orient="columns")
    clear_output(wait=True)
    display(df)
  
  def track_loss(self, loss):
    self.epoch_loss += loss.item() * self.loader.batch_size

  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self.get_num_correct(preds, labels)

  @torch.no_grad()
  def get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

  def save(self, fileName):
    pd.DataFrame.from_dict(self.run_data, orient="columns").to_csv(f"{fileName}.csv")

    with open(f"{fileName}.json", "w", encoding="utf-8") as f:
      json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [31]:
params = OrderedDict(
    lr = [0.001, 0.003],
    batch_size = [1024]
)

run_manager = RunManager()

for run in RunBuilder.get_runs(params):
  network = Network()
  loader = DataLoader(train_set, batch_size=run.batch_size, shuffle=True)
  optimizer = optim.Adam(network.parameters(), lr=run.lr)

  run_manager.begin_run(run, network, loader)
  for epoch in range(20):
    run_manager.begin_epoch()
    for batch in loader:
      images, labels = batch

      preds = network(images)
      loss = F.cross_entropy(preds, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      run_manager.track_loss(loss)
      run_manager.track_num_correct(preds, labels)
    run_manager.end_epoch()
  run_manager.end_run()
run_manager.save("results")

,run,epoch,loss,accuracy,epoch_duration,run_duration,lr,batch_size
0,1,1,1.369948,0.542933,34.291605,34.291611,0.001,1024
1,1,2,0.739713,0.724733,34.012076,68.318216,0.001,1024
2,1,3,0.611465,0.770567,35.195183,103.527538,0.001,1024
3,1,4,0.541582,0.797850,34.013212,137.552859,0.001,1024
4,1,5,0.491806,0.820417,32.810585,170.382180,0.001,1024
5,1,6,0.460201,0.834417,33.288192,203.683328,0.001,1024
6,1,7,0.432333,0.844500,33.273493,236.971959,0.001,1024
7,1,8,0.418974,0.849617,32.780437,269.763561,0.001,1024
8,1,9,0.401485,0.855150,33.876072,303.651075,0.001,1024
9,1,10,0.384300,0.863000,32.409887,336.073388,0.001,1024
